# Trajectron Examples

In [1]:
import torch

from mantrap.environment import Trajectron


trajectron = Trajectron()
trajectron.add_ado(position=torch.zeros(2), velocity=torch.tensor([1, 0]))

ado_trajectories = trajectron.predict_wo_ego(t_horizon=5)

In [11]:
import time
import numpy as np

a = np.random.rand(25) * 100
start_time = time.time()
a_sorted = np.sort(a)
end_time = time.time()
print(end_time - start_time)

start_time = time.time()
ind = a.argsort()
end_time = time.time()
print(end_time - start_time)
print(a[ind])
print(a[ind[::-1]])


0.00021076202392578125
9.918212890625e-05
[ 1.01478948  3.5274069   4.86581533  9.62401784 13.09195919 14.29336559
 15.52399661 31.32223618 35.56511761 36.14662776 40.12382649 40.4625104
 42.57914622 45.12000274 46.50420148 51.34235514 59.4294891  63.86699997
 66.9556088  67.13482245 80.38375926 83.02194236 87.4779835  91.01094223
 92.16425187]
[92.16425187 91.01094223 87.4779835  83.02194236 80.38375926 67.13482245
 66.9556088  63.86699997 59.4294891  51.34235514 46.50420148 45.12000274
 42.57914622 40.4625104  40.12382649 36.14662776 35.56511761 31.32223618
 15.52399661 14.29336559 13.09195919  9.62401784  4.86581533  3.5274069
  1.01478948]
